In [8]:
!pip install nltk


In [9]:
!pip install nltk
import nltk
nltk.download('vader_lexicon')

# Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen, Request
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Parameters
n = 3
tickers = ['AAPL', 'TSLA', 'AMZN']

# Get Data
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'user-agent': 'Mozilla/5.0'})
    resp = urlopen(req)
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')

        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))

        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass

# Iterate through the news
# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        # Check if an <a> tag exists before calling get_text()
        a_tag = x.find('a')
        if a_tag:
            text = a_tag.get_text()
            date_scrape = x.td.text.split()

            if len(date_scrape) == 1:
                # Handle the case where date is "Today"
                if date_scrape[0] == 'Today':
                    time = date_scrape[0]  # Keep 'Today' as is for now
                else:
                    time = date_scrape[0]

            else:
                date = date_scrape[0]
                time = date_scrape[1]

            ticker = file_name.split('_')[0]

            parsed_news.append([ticker, date, time, text])
        else:
            print("Warning: Missing <a> tag in a table row.") # Optional: Print a warning
# Sentiment Analysis
analyzer = SentimentIntensityAnalyzer() # Now you should be able to create the analyzer

columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns=columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix='_right')


# View Data
# Handle "Today" by converting it to the current date
import datetime
today = datetime.date.today()
news['Date'] = news['Date'].replace('Today', today.strftime('%b-%d-%y')) # Replace 'Today' with today's date in the expected format
news['Date'] = pd.to_datetime(news['Date'], format='%b-%d-%y').dt.date # Tell pandas the expected date format

unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

values = []
for ticker in tickers:
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns = ['Headline'])
    print ('\n')
    print (dataframe.head())

    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)

df = pd.DataFrame(list(zip(tickers, values)), columns =['Ticker', 'Mean Sentiment'])
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending=False)
print ('\n')
print (df)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!




Recent News Headlines for AAPL: 
Most top CEOs aren't giving to either Trump or Biden ( Today 04:00AM )
TSMC may raise sales growth forecast for 2024 ( Jul-08-24 09:17PM )
Apple is considering to Revive its plans to make iPads and AirPods MagSafe Charging Cases in Pune, India ( 06:40PM )


Recent News Headlines for TSLA: 
Tesla Stocks Win Streak Hit 9 Days. It Almost Didnt Happen. ( Today 04:10AM )
Most top CEOs aren't giving to either Trump or Biden ( 04:00AM )
BP p.l.c. (BP): Why are Hedge Funds Bullish on This FTSE Dividend Stock Right Now? ( Jul-08-24 11:37PM )


Recent News Headlines for AMZN: 
Will Walmart Be a Trillion-Dollar Stock by 2030? The Answer May Surprise You. ( Jul-08-24 08:00PM )
Why Is Amazon Investing in Saks + Neimans? ( 05:05PM )
How an accident helped the creators of the Mighty Patch unlock Amazon success ( 05:01PM )


              Date     Time    neg    neu    pos  compound
Ticker                                                    
AAPL    2024-07-09  04:00A